In [1]:
import random
import time
import random
import math
import itertools
import numpy as np
import time
from queue import PriorityQueue
from tabulate import tabulate

In [2]:
# to reshape string into matrix
def printStringMatrix(state_type, state_characters):
    print(state_type)
    temp=state_characters[:]
    print(np.reshape([*temp],(3,3)))

# to take start state, target state as input [from user] and store the indexes of respective elements in target state using dictionary
def readInputs():
  sourceCharacters = input('Enter the source_state: ')
  targetCharacters = input('Enter the target_state: ')
  printStringMatrix('Source State', sourceCharacters)
  printStringMatrix('Target State', targetCharacters)

  h2Dictionary = {}
  for e in range(len(targetCharacters)):
    x = int(e/3)
    y = int(e%3)
    h2Dictionary[targetCharacters[e]] = (x,y)
    
  return sourceCharacters, targetCharacters, h2Dictionary

In [3]:
# function to find out the x and y coordinates of the blank element
def initialBlankCoordinates(sourceCharacters):
    for i in range(len(sourceCharacters)):
        if sourceCharacters[i] == 'B':
            return i

    return -1

In [4]:
# accessing elements at (x-1,y), (x+1,y), (x,y-1) and (x,y+1) unless invalid
def goUp(idx):
    if idx > 2:
        return idx - 3
    return -1

def goDown(idx):
    if idx < 6:
        return idx + 3
    return -1

def goLeft(idx):
    if idx != 0 and idx != 3 and idx != 6:
        return idx -1
    return -1

def goRight(idx):
    if idx !=2 and idx != 5 and idx != 8:
        return idx + 1
    return -1

# traversing to valid states
def traverse(currentBlank, currentState):
  commands = [goRight,goLeft,goUp,goDown]  
  validNextStates = []
  for e in range(len(commands)):
    idx = commands[e](currentBlank)
    if idx != -1:
      cpy = currentState[:]
      newBlank = idx
      newBlankCharacter = cpy[newBlank]
      cpy = cpy[:currentBlank] + cpy[newBlank] + cpy[currentBlank+1:]
      cpy = cpy[:newBlank] + "B" + cpy[newBlank+1:]
      validNextStates.append((cpy,newBlank))
      
  return validNextStates

In [5]:
# class for State [current state, g(n), h(n), coordiantes of blank element]
class Priority_State:
    def __init__(self, _state, g_n, h_n, _blank):
        self.state = _state 
        self.g_n = g_n
        self.h_n = h_n
        self.blank_state = _blank

# for final output table
class HeuristicInformation_State:
      def __init__(self, algorithm, cooling_function,optimalPathCost, optimalPath,optimalPathPrint, parentList, timeTaken, reachable):
        self.algorithm = algorithm 
        self.cooling_function=cooling_function
        self.optimalPathCost = optimalPathCost
        self.optimalPath = optimalPath
        self.optimalPathPrint = optimalPathPrint
        self.parentList = parentList
        self.timeTaken = timeTaken
        self.reachable = reachable

In [6]:
# to call correct heuristic functions
def heuristic(op, intermediateCharacters, targetCharacters, h2Dictionary):
  if op == 1:
    return h1Heuristic(intermediateCharacters, targetCharacters)
  if op == 2:
    return h2Heuristic(intermediateCharacters, targetCharacters, h2Dictionary)
  if op == 3:
    return h3Heuristic(intermediateCharacters, targetCharacters, h2Dictionary)

# Heuristic 1-> h(n)= Tiles displaced ignoring Blank character tile
def h1Heuristic(intermediateCharacters, targetCharacters):
  cnt = 0
  for e in range(len(targetCharacters)):
    if intermediateCharacters[e] != 'B' and intermediateCharacters[e] != targetCharacters[e]:
      cnt = cnt + 1
  return cnt

# Heuristic 2-> h(n)= Manhatten distance ignoring blank character tile
def h2Heuristic(intermediateCharacters, targetCharacters, h2Dictionary):
  dis = 0
  for e in range(len(intermediateCharacters)):
    if intermediateCharacters[e] != 'B':
      x = int(e / 3)
      y = e % 3
      # Index of intermediateCharacters[e] in target
      _x = h2Dictionary[intermediateCharacters[e]][0]
      _y = h2Dictionary[intermediateCharacters[e]][1]
      dis = dis + abs(x -_x) + abs(y -_y)
  return dis

# Heuristic 3-> h3(n)= h1(n)*h2(n)
def h3Heuristic(intermediateCharacters, targetCharacters, h2Dictionary):
  dis=h1Heuristic(intermediateCharacters, targetCharacters)*h2Heuristic(intermediateCharacters, targetCharacters, h2Dictionary)
  return dis

In [7]:
# finding final path taken
def constructOptimalPath(parent_list, sourceCharacters, targetCharacters):
    currentCharacters = targetCharacters
    optimalPath = []
    while currentCharacters != sourceCharacters:
        optimalPath.append(currentCharacters)
        currentCharacters = parent_list[currentCharacters]   
    optimalPath.append(sourceCharacters)   
    return optimalPath,printOptimalPath(optimalPath)

# printing path
def printOptimalPath(optimalPath):
    optimalPathPrint = ''
    for e in range(len(optimalPath) - 1, -1, -1):
        optimalPathPrint = optimalPathPrint + optimalPath[e] + '->'
    return optimalPathPrint[:-2]

# function to construct final output table 
def constructTable(algorithm_result):
  headerData = ['Algorithm','Cooling Function', 'Path Cost','Final state reached', 'Path States', 'Execution Time', 'Reachable',  'Path Traversed']
  tableData = []
  for h in algorithm_result:
    rowValues = []
    rowValues.append(h.algorithm) 
    rowValues.append(h.cooling_function)
    rowValues.append(h.optimalPathCost) 
    rowValues.append(h.optimalPath[0])
    rowValues.append(len(h.optimalPath)) 
    rowValues.append(h.timeTaken)
    rowValues.append(h.reachable)
    rowValues.append(h.optimalPathPrint)
    tableData.append(rowValues)
  print(tabulate(tableData, headers=headerData, tablefmt="grid"))

# for reachable/ unreachable states
def analyseAlgorithms(algorithm_result):
  constructTable(algorithm_result)
  return

In [8]:
def get_temperature(max_temperature, iteration, choice):
    if choice == 1:
        return max_temperature*(0.95**iteration)
    elif choice == 2:
        return max_temperature/iteration
    elif choice == 3:
        if iteration == 1:
            return max_temperature
        return max_temperature / math.log(iteration)

In [9]:
# Simulated Annealing function
def SimulatedAnnealing(op, sourceCharacters, targetCharacters,idx,cooling_function, h2Dictionary = {}):   
    
    # Parameters for simulated annealing
    max_iterations = 5*10**4
    max_temperature = 5*10**4

    # for final path
    parent_list = {}
    parent_list[sourceCharacters]=sourceCharacters

    # to keep track of visited states
    visitedDict = {}

    # manage current iteration
    current_iteration = 0
  
    # current state
    currentState = Priority_State(sourceCharacters, 0, heuristic(op, sourceCharacters, targetCharacters, h2Dictionary), idx)

    # traversing to neighbours
    while max_iterations > 0:

      # if target reached
      if currentState.state == targetCharacters:
        return parent_list, True, currentState.g_n, currentState

      current_iteration = current_iteration + 1
      max_iterations = max_iterations - 1

      # mark current state visited
      visitedDict[currentState.state] = 1

      temp = get_temperature(max_temperature, current_iteration, cooling_function)
      if temp == 0: 
        temp = 1

      current_cost = currentState.h_n

      # find index of current blank tile
      currentBlank = currentState.blank_state

      # holds all valid next neighbours
      validNextStates = traverse(currentBlank, currentState.state)
      # shuffle neighbors
      random.shuffle(validNextStates)

      # finding next neighbour
      nextState = currentState

      for neighbourCharacters,neighbor_blank in validNextStates:
        if visitedDict.get(neighbourCharacters) != None:
          continue
        neighbour_cost = heuristic(op,neighbourCharacters,targetCharacters,h2Dictionary)
        probability=0
        if neighbour_cost < current_cost:
          probability = 1
        else:
          probability =math.e ** (-1*(neighbour_cost - current_cost) / temp)
        r=random.random()
        if r<= probability:
          nextState=Priority_State(neighbourCharacters, currentState.g_n+1, neighbour_cost,neighbor_blank)
          parent_list[neighbourCharacters]=currentState.state
        break
  
      currentState=nextState

    return parent_list, False, currentState.g_n, currentState

In [10]:
# utility function for calling Simulated Annealing
def runSimulatedAnnealing(op, sourceCharacters, targetCharacters,idx, cooling_function,algorithm, h2Dictionary = {}):
  start_time = time.time()
  parent_list, result, optimalPathCost, reachedTarget = SimulatedAnnealing(op, sourceCharacters, targetCharacters, idx, cooling_function,h2Dictionary)
  optimalPath, optimalPathPrint = constructOptimalPath(parent_list, sourceCharacters, reachedTarget.state)
  end_time = time.time()
  time_taken = end_time - start_time

  return HeuristicInformation_State(algorithm,cooling_function, optimalPathCost, optimalPath, optimalPathPrint, parent_list, time_taken, result)

In [11]:
# calling function for the initial matrix, target matrix and matrix- characters
sourceCharacters, targetCharacters, h2Dictionary= readInputs()
# index of current blank element
idx  = initialBlankCoordinates(sourceCharacters)

SimulatedAnnealing_results = []
SimulatedAnnealing_results.append(runSimulatedAnnealing(1,sourceCharacters,targetCharacters,idx,1,'h1(n)',h2Dictionary))
SimulatedAnnealing_results.append(runSimulatedAnnealing(1,sourceCharacters,targetCharacters,idx,2,'h1(n)',h2Dictionary))
SimulatedAnnealing_results.append(runSimulatedAnnealing(1,sourceCharacters,targetCharacters,idx,3,'h1(n)',h2Dictionary))
SimulatedAnnealing_results.append(runSimulatedAnnealing(2,sourceCharacters,targetCharacters,idx,1,'h2(n)',h2Dictionary))
SimulatedAnnealing_results.append(runSimulatedAnnealing(2,sourceCharacters,targetCharacters,idx,2,'h2(n)',h2Dictionary))
SimulatedAnnealing_results.append(runSimulatedAnnealing(2,sourceCharacters,targetCharacters,idx,3,'h2(n)',h2Dictionary))

analyseAlgorithms(SimulatedAnnealing_results)

# Enter the start_state: B23145678, 
# Enter the target_state: 2431B5678  (TRUE) and 12345867B (FALSE)
# 7B2453681 

# 4176B2358 
# 12345678B

Enter the source_state: 12345678B
Enter the target_state: 123456B78
Source State
[['1' '2' '3']
 ['4' '5' '6']
 ['7' '8' 'B']]
Target State
[['1' '2' '3']
 ['4' '5' '6']
 ['B' '7' '8']]
+-------------+--------------------+-------------+-----------------------+---------------+------------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------